In [1]:
import pymysql
import requests

#這裡的環境用.venv 虛擬環境

##### 下載分析pm2.5 opendata資料

In [2]:
url="https://data.moenv.gov.tw/api/v2/aqx_p_02?api_key=540e2ca4-41e1-4186-8497-fdd67024ac44&limit=1000&sort=datacreationdate%20desc&format=JSON" 
url

'https://data.moenv.gov.tw/api/v2/aqx_p_02?api_key=540e2ca4-41e1-4186-8497-fdd67024ac44&limit=1000&sort=datacreationdate%20desc&format=JSON'

In [3]:
resp=requests.get(url,verify=False)
resp

/Users/songsiying/Desktop/📖我的進修生涯📖/聯成-python/6.Django1140716/7.1140820-pm25_tomysql/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.moenv.gov.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>

In [4]:
datas=resp.json()['records']
datas


[{'site': '員林',
  'county': '彰化縣',
  'pm25': '18',
  'datacreationdate': '2025-08-21 22:00',
  'itemunit': 'μg/m3'},
 {'site': '大城',
  'county': '彰化縣',
  'pm25': '10',
  'datacreationdate': '2025-08-21 22:00',
  'itemunit': 'μg/m3'},
 {'site': '富貴角',
  'county': '新北市',
  'pm25': '9',
  'datacreationdate': '2025-08-21 22:00',
  'itemunit': 'μg/m3'},
 {'site': '麥寮',
  'county': '雲林縣',
  'pm25': '8',
  'datacreationdate': '2025-08-21 22:00',
  'itemunit': 'μg/m3'},
 {'site': '關山',
  'county': '臺東縣',
  'pm25': '2',
  'datacreationdate': '2025-08-21 22:00',
  'itemunit': 'μg/m3'},
 {'site': '馬公',
  'county': '澎湖縣',
  'pm25': '0',
  'datacreationdate': '2025-08-21 22:00',
  'itemunit': 'μg/m3'},
 {'site': '金門',
  'county': '金門縣',
  'pm25': '8',
  'datacreationdate': '2025-08-21 22:00',
  'itemunit': 'μg/m3'},
 {'site': '馬祖',
  'county': '連江縣',
  'pm25': '8',
  'datacreationdate': '2025-08-21 22:00',
  'itemunit': 'μg/m3'},
 {'site': '埔里',
  'county': '南投縣',
  'pm25': '15',
  'datacreationdat

In [5]:
len(datas)

1000

In [6]:
datas[0]

{'site': '員林',
 'county': '彰化縣',
 'pm25': '18',
 'datacreationdate': '2025-08-21 22:00',
 'itemunit': 'μg/m3'}

### 建立資料表

In [7]:
#按上面提示去打，除了id自己新增的
table_str="""
create table if not exists pm25(
id int auto_increment primary key,
site varchar(25),
county varchar(50),
pm25 int,
datacreationdate datetime,
itemunit varchar(20),
unique key site_time (site,datacreationdate)
)
"""
print(table_str)


create table if not exists pm25(
id int auto_increment primary key,
site varchar(25),
county varchar(50),
pm25 int,
datacreationdate datetime,
itemunit varchar(20),
unique key site_time (site,datacreationdate)
)



##### 連接資料庫

In [8]:
conn=pymysql.connect(
host="localhost",
user="root",
password="",
port=3306,
database="demo"
)

cursor=conn.cursor()
cursor

In [9]:
cursor.execute(table_str)
conn.commit()

##### 寫入一筆資料

In [10]:
#字串太長可用\區隔 #資料表的名稱pm25也要寫入
sqlstr="insert into pm25(site,county,pm25,datacreationdate,itemunit)\
      values('{}','{}',{},'{}','{}')"

data=list(datas[0].values())
print(data)
sqlstr.format(data[0],data[1],data[2],data[3],data[4])

['員林', '彰化縣', '18', '2025-08-21 22:00', 'μg/m3']


"insert into pm25(site,county,pm25,datacreationdate,itemunit)      values('員林','彰化縣',18,'2025-08-21 22:00','μg/m3')"

In [11]:
cursor.execute(sqlstr.format(data[0],data[1],data[2],data[3],data[4]))
conn.commit()

##### 一次寫入
- 加上 ignore

In [12]:
sqlstr="insert ignore into pm25(site,county,pm25,datacreationdate,itemunit)\
      values(%s,%s,%s,%s,%s)"

# 移除pm25空字串部分
values=[list(data.values()) for data in datas if list(data.values())[2]!=""]

cursor.executemany(sqlstr,values)
conn.commit()

In [13]:
conn.close()

In [93]:
pip install pandas
import pandas as pd

SyntaxError: invalid syntax (3716164084.py, line 1)

In [91]:
cursor.execute("select * from pm25")

datas=cursor.fetchall #所有資料
datas

InterfaceError: (0, '')

In [ ]:
df=pd.DataFrame(datas,columns="id","site","county","datetime","unit")
df

In [ ]:
df.groupby("county").get_group("新北市").drop

In [ ]:
df[[""site ","datetime"]].drop_duplicates()]